In [1]:
#export
from exp.nb_03 import *

### DataBunch/Learner

In [2]:
x_train,y_train,x_valid,y_valid = get_data()
train_ds,valid_ds = Dataset(x_train,y_train), Dataset(x_valid,y_valid)
nh,bs = 50,64
c=y_train.max().item()+1
loss_func=F.cross_entropy

In [3]:
#export
class DataBunch():
    def __init__(self, train_dl, valid_dl, c=None):
        self.train_dl,self.valid_dl,self.c = train_dl,valid_dl,c
        
    @property
    def train_ds(self): return self.train_dl.dataset
    
    @property
    def valid_ds(self): return self.valid_dl.dataset

In [4]:
data = DataBunch(*get_dls(train_ds, valid_ds, bs),c)

In [5]:
#export
def get_model(data, lr=0.5, nh=50):
    m = data.train_ds.x.shape[1]
    model = nn.Sequential(nn.Linear(m,nh),nn.ReLU(),nn.Linear(nh,c))
    return model, optim.SGD(model.parameters(),lr=lr)

class Learner():
    def __init__(self, model, opt, loss_func, data):
        self.model,self.opt,self.loss_func,self.data=model, opt, loss_func, data
    

In [6]:
learner = Learner(*get_model(data),loss_func,data)

In [7]:
def fit(epochs,learn):
    for epoch in range(epochs):
        learn.model.train()
        for xb,yb in learn.data.train_dl:
            loss = learn.loss_func(learn.model(xb), yb)
            loss.backward()
            learn.opt.step()
            learn.opt.zero_grad()
            
        learn.model.eval()
        with torch.no_grad():
            tot_loss, tot_acc = 0.,0.
            for xb,yb in learn.data.valid_dl:
                pred = learn.model(xb)
                tot_loss += learn.loss_func(pred,yb)
                tot_acc += accuracy(pred, yb)
        nv = len(learn.data.valid_dl)
        print(epoch,tot_loss/nv,tot_acc/nv)
    return tot_loss/nv, tot_acc/nv

In [9]:
loss,acc=fit(5,learner)

0 tensor(0.1346) tensor(0.9606)
1 tensor(0.1183) tensor(0.9668)
2 tensor(0.1023) tensor(0.9708)
3 tensor(0.1186) tensor(0.9659)
4 tensor(0.1038) tensor(0.9697)
